# Projeto Final – Uber NYC 2014  
## Notebook: Coleta, Limpeza e EDA com KaggleHub (Semana 2)

Este notebook realiza:

1. Autenticação com o **KaggleHub**  
2. Download do dataset **Uber Pickups in New York City (FiveThirtyEight)**  
3. Seleção dos arquivos de **abril a setembro de 2014**  
4. Consolidação em um único `DataFrame`  
5. Limpeza e criação de atributos (`hour`, `weekday`, `month`)  
6. Análise exploratória inicial (EDA)  
7. Sugestões de prompts para geração de insights com IA

> **Importante:** este notebook foi preparado para ser executado no **Google Colab**.


## 1. Instalação de dependências


In [ ]:
# Se estiver no Google Colab, descomente a linha abaixo na primeira execução:
# !pip install kagglehub pandas numpy matplotlib seaborn --quiet


## 2. Imports e configuração inicial


In [ ]:
import os
from pathlib import Path
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)

DATA_DIR = Path("data_uber_raw")
DATA_DIR.mkdir(exist_ok=True)

DATA_DIR


## 3. Download do dataset com KaggleHub


In [ ]:
# Baixa o dataset do Kaggle usando o KaggleHub
# Doc oficial: https://www.kaggle.com/docs/api#kagglehub

try:
    path = kagglehub.dataset_download("fivethirtyeight/uber-pickups-in-new-york-city")
    print("Dataset baixado em:", path)
except Exception as e:
    print("[ERRO] Falha no download pelo KaggleHub.")
    print("Verifique se suas credenciais do Kaggle estão configuradas corretamente.")
    raise e


## 4. Seleção dos arquivos (abril–setembro 2014)


In [ ]:
raw_path = Path(path)

# Lista todos os CSVs baixados
all_csv = sorted(raw_path.glob("*.csv"))
all_csv


In [ ]:
# Filtrar arquivos que contenham 'apr', 'may', 'jun', 'jul', 'aug', 'sep' no nome

target_months = ["apr", "may", "jun", "jul", "aug", "sep"]
selected_files = [f for f in all_csv if any(m in f.name.lower() for m in target_months)]

print("Arquivos selecionados:")
for f in selected_files:
    print(" -", f.name)

if not selected_files:
    raise FileNotFoundError(
        "Nenhum arquivo de abril a setembro foi encontrado. "
        "Verifique os nomes dos arquivos baixados."
    )


## 5. Leitura e concatenação dos CSVs


In [ ]:
dfs = []
for f in selected_files:
    print("Lendo:", f.name)
    tmp = pd.read_csv(f, encoding="utf-8", on_bad_lines="skip")
    dfs.append(tmp)

df_raw = pd.concat(dfs, ignore_index=True)
df_raw.head()


In [ ]:
df_raw.info()


## 6. Limpeza, padronização e criação de atributos


In [ ]:
# Cópia para não mexer no bruto
df = df_raw.copy()

# Padronizar nomes de colunas
df.columns = [c.strip().lower() for c in df.columns]

rename_map = {}
for c in df.columns:
    if "date" in c and "time" in c:
        rename_map[c] = "datetime"
    elif c.lower() == "lat":
        rename_map[c] = "lat"
    elif c.lower() == "lon":
        rename_map[c] = "lon"
    elif c.lower() == "base":
        rename_map[c] = "base"

df = df.rename(columns=rename_map)

# Manter apenas as colunas que interessam
df = df[["datetime", "lat", "lon", "base"]]

# Converter tipos
df["datetime"] = pd.to_datetime(df["datetime"], errors="coerce")

# Remover linhas inválidas e duplicadas
df = df.dropna(subset=["datetime", "lat", "lon", "base"]).drop_duplicates()

# Criar atributos derivados
df["hour"] = df["datetime"].dt.hour
df["weekday"] = df["datetime"].dt.day_name()
df["month"] = df["datetime"].dt.month_name()

df.head()


In [ ]:
df.describe(include="all")


## 7. EDA – Análise Exploratória Inicial


In [ ]:
order_weekday = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

plt.figure()
sns.countplot(data=df, x="weekday", order=order_weekday)
plt.xticks(rotation=45)
plt.title("Número de corridas por dia da semana")
plt.tight_layout()
plt.show()


In [ ]:
plt.figure()
sns.countplot(data=df, x="hour")
plt.title("Número de corridas por hora do dia")
plt.tight_layout()
plt.show()


In [ ]:
pivot = df.pivot_table(
    index="weekday",
    columns="hour",
    values="base",
    aggfunc="count"
).reindex(order_weekday)

plt.figure(figsize=(12, 6))
sns.heatmap(pivot, cmap="coolwarm")
plt.title("Heatmap de corridas por dia da semana e hora")
plt.ylabel("Dia da semana")
plt.xlabel("Hora do dia")
plt.tight_layout()
plt.show()


## 8. Sugestões de prompts para IA

**Prompt 1 – Resumo geral da EDA**  
> Você é um analista de dados. Com base nos gráficos gerados sobre corridas da Uber em NYC (por dia da semana, por hora do dia e no heatmap weekday × hour), escreva um resumo de até 10 linhas explicando os principais padrões encontrados, horários de pico e possíveis interpretações.

**Prompt 2 – Foco em operações e negócios**  
> Atue como consultor de negócios para mobilidade urbana. A partir dos padrões de corridas da Uber por dia da semana e por hora do dia, elabore recomendações para alocação de motoristas, definição de tarifas dinâmicas e campanhas promocionais em horários de baixo movimento.

**Prompt 3 – Geração de hipóteses**  
> Com base nas distribuições de corridas por hora e por dia da semana, gere de 5 a 8 hipóteses que poderiam ser testadas em análises futuras. As hipóteses devem relacionar comportamento de demanda, eventos sazonais, clima ou perfil dos usuários.
